In [2]:
# code for loading the format for the notebook
import os

# path : store the current path to convert back to it later
path = os.getcwd()
os.chdir(os.path.join('..', '..', 'notebook_format'))
from formats import load_style
load_style(plot_style = False)

In [3]:
os.chdir(path)
import pandas as pd
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession

# create the SparkSession class,
# which is the entry point into all functionality in Spark
# The .master part sets it to run on all cores on local, note
# that we should leave out the .master part if we're actually
# running the job on a cluster, or else we won't be actually
# using the cluster
spark = (SparkSession
         .builder
         .master('local[*]')
         .appName('Crime')
         .config(conf = SparkConf())
         .getOrCreate())

# set the log level to ERROR to prevent 
# the terminal from showing too many information
sc = spark.sparkContext
sc.setLogLevel('ERROR')

# 1. magic for inline plot
# 2. magic to print version
# 3. magic so that the notebook will reload external python modules
%matplotlib inline
%load_ext watermark
%load_ext autoreload 
%autoreload 2

%watermark -a 'Ethen' -d -t -v -p numpy,pandas,matplotlib,pyspark

Ethen 2017-09-16 21:15:35 

CPython 3.5.2
IPython 6.1.0

numpy 1.13.1
pandas 0.20.3
matplotlib 2.0.0
pyspark 2.2.0


In [3]:
filepath = "/Users/ethen/programming/advanced_analytics_spark/ch02-intro/linkage/*.csv"
parsed = (spark.read.
    option("header", "true").
    option("nullValue", "?").
    option("inferSchema", "true").
    csv(filepath))
parsed.show(5)

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| 3148| 8326|           1|        null|           1|        null|      1|     1|     1|     1|      1|    true|
|14055|94934|           1|        null|           1|        null|      1|     1|     1|     1|      1|    true|
|33948|34740|           1|        null|           1|        null|      1|     1|     1|     1|      1|    true|
|  946|71870|           1|        null|           1|        null|      1|     1|     1|     1|      1|    true|
|64880|71676|           1|        null|           1|        null|      1|     1|     1|     1|      1|    true|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--

In [4]:
(parsed.
    groupBy("is_match").
    count().
    orderBy(F.desc("count")).
    show())

+--------+-------+
|is_match|  count|
+--------+-------+
|   false|5728201|
|    true|  20931|
|    null|      1|
+--------+-------+



In [5]:
parsed.createOrReplaceTempView("linkage")
spark.sql("""
    SELECT is_match, COUNT(*) AS count
    FROM linkage
    GROUP BY is_match
    ORDER BY count DESC
""").show()

+--------+-------+
|is_match|  count|
+--------+-------+
|   false|5728201|
|    true|  20931|
|    null|      1|
+--------+-------+



In [6]:
summary = parsed.describe()
summary.select("summary", "cmp_fname_c1", "cmp_fname_c2").show()

+-------+--------------------+-------------------+
|summary|        cmp_fname_c1|       cmp_fname_c2|
+-------+--------------------+-------------------+
|  count|             5748126|             103699|
|   mean|  0.7129023464249419|  0.900008998936421|
| stddev| 0.38875843950829186|0.27133067681523776|
|    min|                   0|                  0|
|    max|2.68694413843136e-05|                  1|
+-------+--------------------+-------------------+



In [7]:
matches = parsed.where("is_match = true")
matches_summary = matches.describe()

misses = parsed.filter(F.col("is_match") == False)
misses_summary = misses.describe()

In [21]:
df_matches = matches_summary.toPandas()
df_matches

,summary,id_1,id_2,cmp_fname_c1,cmp_fname_c2,cmp_lname_c1,cmp_lname_c2,cmp_sex,cmp_bd,cmp_bm,cmp_by,cmp_plz
0,count,20931,20931,20922,1333,20931,475,20931,20925,20925,20925,20902
1,mean,34575.72117911232,51259.95939037791,0.9973163859635038,0.9898900320318176,0.9970152595958817,0.969370167843852,0.987291577086618,0.9970848267622461,0.9979450418160095,0.9961290322580645,0.9584250310975027
2,stddev,21950.31285196913,24345.73345377519,0.03650667584833679,0.08251973727615237,0.043118807533945126,0.15345280740388917,0.11201570591216435,0.05391487659807981,0.045286127452170664,0.06209804856731055,0.19962063345931919
3,min,10001,10010,0,0,0,0,0,0,0,0,0
4,max,99946,99996,1,1,1,1,1,1,1,1,1


In [34]:
# pivot to a wide format
colnames = ['field']
colnames.extend(df_matches['summary'])
df_wide = df_matches.T.iloc[1:].reset_index()
df_wide.columns = colnames
df_wide.head()

,field,count,mean,stddev,min,max
0,id_1,20931,34575.72117911232,21950.31285196913,10001,99946
1,id_2,20931,51259.95939037791,24345.73345377519,10010,99996
2,cmp_fname_c1,20922,0.9973163859635038,0.03650667584833679,0,1
3,cmp_fname_c2,1333,0.9898900320318176,0.08251973727615237,0,1
4,cmp_lname_c1,20931,0.9970152595958817,0.043118807533945126,0,1
